### KNN - Predicting Electricity Use for UES-Seacoast Service Territory with 10 Years of ISO & UES Data
##### NO WEATHER DATA USED!!!

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

### Predict UES load - 1 Day Ahead

In [2]:
# Read in data
data = pd.read_csv("ISO.aggregate - 1 day SHIFTED.csv")

In [3]:
# See first few rows of data
data.head()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.NEXT.DAY.LOAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh,Wk.Day
0,1,1,2010,1,13797.0,63.04,63.33,30.142,67.858,61.904,1168.0,1119.7,640.4,3206.8,825.4,1498.2,1774.3,2688.9,Fri
1,1,1,2010,2,13560.0,57.61,69.74,27.370,63.276,58.754,1091.8,1077.1,605.4,3031.6,781.9,1404.3,1676.8,2569.0,Fri
2,1,1,2010,3,13121.0,53.74,55.39,25.573,59.899,57.320,1046.4,1045.1,578.9,2902.9,749.7,1340.0,1621.8,2483.2,Fri
3,1,1,2010,4,13134.0,42.09,60.49,24.728,58.275,57.068,1021.5,1030.5,556.9,2827.7,729.7,1306.1,1588.5,2428.1,Fri
4,1,1,2010,5,13144.0,47.95,53.43,24.974,58.516,57.858,1018.0,1047.4,553.7,2820.7,730.0,1308.1,1586.7,2432.2,Fri


In [4]:
# Check dataframe data types
data.dtypes

Month                      int64
Day                        int64
Year                       int64
TI                         int64
ISO.tot.dmd              float64
LMP.DA.NH                float64
LMP.RT.NH                float64
UES.Cap                  float64
UES.Sea                  float64
UES.Sea.NEXT.DAY.LOAD    float64
NH.RT.MWh                float64
ME.RT.MWh                float64
VT.RT.MWh                float64
CT.RT.MWh                float64
RI.RT.MWh                float64
MA.SE.RT.MWh             float64
MA.WC.RT.MWh             float64
MA.NE.RT.MWh             float64
Wk.Day                    object
dtype: object

In [5]:
# Summary Stats
data.describe()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.NEXT.DAY.LOAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh
count,78864.000000,78864.000000,78864.000000,78864.00000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000,78864.000000
mean,6.522520,15.723182,2013.998783,12.50000,14580.600875,44.318309,43.562332,49.924784,76.881439,76.881482,1319.449053,1306.260874,644.164399,3482.634094,928.737496,1689.311137,1962.395218,2885.571864
std,3.447958,8.797753,2.581141,6.92223,2701.434331,34.645532,41.976925,16.058602,18.113627,18.113338,265.309210,205.406096,107.807006,771.813292,206.087265,389.960098,378.896406,570.384539
min,1.000000,1.000000,2010.000000,1.00000,8665.000000,-5.000000,-155.110000,5.395000,6.912000,6.912000,523.200000,681.800000,302.300000,1367.400000,371.800000,708.700000,736.900000,1729.700000
25%,4.000000,8.000000,2012.000000,6.75000,12636.022500,26.567500,24.197500,37.744000,64.281750,64.283500,1121.200000,1143.500000,558.900000,2937.975000,784.100000,1413.275000,1683.000000,2473.475000
50%,7.000000,16.000000,2014.000000,12.50000,14396.130000,36.180000,34.460000,49.499000,76.038000,76.038000,1333.100000,1329.600000,648.800000,3444.300000,916.600000,1659.400000,1951.600000,2869.400000
75%,10.000000,23.000000,2016.000000,18.25000,16121.000000,48.680000,47.760000,60.214000,86.302250,86.300250,1485.400000,1450.125000,721.500000,3915.500000,1032.300000,1887.400000,2199.025000,3210.900000
max,12.000000,31.000000,2018.000000,24.00000,27762.000000,785.000000,2493.150000,118.547000,168.430000,168.430000,2433.300000,2034.100000,1002.500000,7218.800000,4314.300000,3644.700000,3652.300000,5657.900000


In [6]:
# Shape of dataframe
data.shape

(78864, 19)

In [7]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                      int64
Day                        int64
Year                       int64
TI                         int64
ISO.tot.dmd              float64
LMP.DA.NH                float64
LMP.RT.NH                float64
UES.Cap                  float64
UES.Sea                  float64
UES.Sea.NEXT.DAY.LOAD    float64
NH.RT.MWh                float64
ME.RT.MWh                float64
VT.RT.MWh                float64
CT.RT.MWh                float64
RI.RT.MWh                float64
MA.SE.RT.MWh             float64
MA.WC.RT.MWh             float64
MA.NE.RT.MWh             float64
Wk.Day_Fri                 uint8
Wk.Day_Mon                 uint8
Wk.Day_Sat                 uint8
Wk.Day_Sun                 uint8
Wk.Day_Thu                 uint8
Wk.Day_Tue                 uint8
Wk.Day_Wed                 uint8
dtype: object

In [8]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.NEXT.DAY.LOAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.NEXT.DAY.LOAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [9]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [10]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59148, 24)
Training Target Shape: (59148,)
Testing Predictors Shape: (19716, 24)
Testing Target Shape: (19716,)


In [11]:
# Create scaler
scaler = StandardScaler()

# Scale training predictors
train_pred_scaled = scaler.fit_transform(train_predictors)

# Scale test predictors
test_pred_scaled = scaler.transform(test_predictors)

# Create KNN model
model = KNeighborsRegressor()

# Train model with scaled training data and target values
model.fit(train_pred_scaled, train_target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [12]:
# Calculate errors for training data using mean squared error (MSE) and mean absolute error (MAE)
mse = mean_squared_error(train_target, model.predict(train_pred_scaled))
mae = mean_absolute_error(train_target, model.predict(train_pred_scaled))

In [13]:
# Display training MSE, MAE, & RMSE
print("mse = ",mse," & mae = ",mae," & rmse = ", sqrt(mse))

mse =  13.020136968452693  & mae =  2.2959167545817274  & rmse =  3.60834268999671


In [14]:
# Calculate and display MSE, MAE, & RMSE on test data
test_mse = mean_squared_error(test_target, model.predict(test_pred_scaled))
test_mae = mean_absolute_error(test_target, model.predict(test_pred_scaled))
print("mse = ",test_mse," & mae = ",test_mae," & rmse = ", sqrt(test_mse))

mse =  22.186856271746805  & mae =  3.054505498072631  & rmse =  4.710292588762062


In [15]:
# Use predict method on the test data
predictions = model.predict(test_pred_scaled)

# Calculate max and mean test_target value
test_target_max_1 = max(test_target)
test_target_mean_1 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_1 = max(errors)
mean_absolute_error_1 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_1 = 100 * (errors / test_target)
accuracy_1 = 100 - np.mean(mape_1)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_1, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_1, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_1, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_1, 2), 'MW')
print('Accuracy:', round(accuracy_1, 2), '%.')

Target Variable Max Value: 166.65 MW
Max Absolute Error: 42.61 MW
Mean Target Variable Value: 77.05 MW
Mean Absolute Error: 3.05 MW
Accuracy: 96.09 %.


In [16]:
# Grid search for KNN to see if performance can be improved
grid_params = {
    'n_neighbors': [3,5,11,19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

gs = GridSearchCV(
     KNeighborsRegressor(),
     grid_params,
     verbose = 1,
     cv = 3,
     n_jobs = -1
)

gs_knn = gs.fit(train_pred_scaled, train_target)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 21.5min finished


In [17]:
gs_knn.best_score_

0.9335710132767716

In [18]:
gs_knn.best_estimator_

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='manhattan',
          metric_params=None, n_jobs=None, n_neighbors=3, p=2,
          weights='distance')

In [19]:
gs_knn.best_params_

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}

In [20]:
# See if the grid search improved the accuracy of KNN
best_grid = gs_knn.best_estimator_

# Use predict method on the test data
predictions = best_grid.predict(test_pred_scaled)

# Calculate max and mean test_target value
test_target_max_GS_1 = max(test_target)
test_target_mean_GS_1 = np.mean(test_target)

# Calculate the absolute errors
errors = abs(predictions - test_target)
max_absolute_error_GS_1 = max(errors)
mean_absolute_error_GS_1 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_GS_1 = 100 * (errors / test_target)
accuracy_GS_1 = 100 - np.mean(mape_GS_1)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_GS_1, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_GS_1, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_GS_1, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_GS_1, 2), 'MW')
print('Accuracy:', round(accuracy_GS_1, 2), '%.')

Target Variable Max Value: 166.65 MW
Max Absolute Error: 41.26 MW
Mean Target Variable Value: 77.05 MW
Mean Absolute Error: 2.49 MW
Accuracy: 96.79 %.


### Predict UES load - 2 Days Ahead

In [21]:
# Read in data
data = pd.read_csv("ISO.aggregate - 2 day SHIFTED.csv")

In [22]:
# See first few rows of data
data.head()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.2.DAYS.AHEAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh,Wk.Day
0,1,1,2010,1,13797.0,63.04,63.33,30.142,67.858,67.512,1168.0,1119.7,640.4,3206.8,825.4,1498.2,1774.3,2688.9,Fri
1,1,1,2010,2,13560.0,57.61,69.74,27.370,63.276,64.690,1091.8,1077.1,605.4,3031.6,781.9,1404.3,1676.8,2569.0,Fri
2,1,1,2010,3,13121.0,53.74,55.39,25.573,59.899,63.511,1046.4,1045.1,578.9,2902.9,749.7,1340.0,1621.8,2483.2,Fri
3,1,1,2010,4,13134.0,42.09,60.49,24.728,58.275,63.021,1021.5,1030.5,556.9,2827.7,729.7,1306.1,1588.5,2428.1,Fri
4,1,1,2010,5,13144.0,47.95,53.43,24.974,58.516,63.480,1018.0,1047.4,553.7,2820.7,730.0,1308.1,1586.7,2432.2,Fri


In [23]:
# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.2.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day                   object
dtype: object

In [24]:
# Summary Stats
data.describe()

,Month,Day,Year,TI,ISO.tot.dmd,LMP.DA.NH,LMP.RT.NH,UES.Cap,UES.Sea,UES.Sea.2.DAYS.AHEAD,NH.RT.MWh,ME.RT.MWh,VT.RT.MWh,CT.RT.MWh,RI.RT.MWh,MA.SE.RT.MWh,MA.WC.RT.MWh,MA.NE.RT.MWh
count,78840.000000,78840.000000,78840.000000,78840.00000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000,78840.000000
mean,6.520852,15.718836,2013.997565,12.50000,14581.031134,44.322627,43.568269,49.929284,76.881728,76.880756,1319.456216,1306.235604,644.156854,3482.736310,928.771856,1689.358879,1962.457955,2885.665101
std,3.447158,8.795564,2.580589,6.92223,2701.598371,34.649625,41.981423,16.058460,18.115293,18.114217,265.333515,205.410929,107.814199,771.876639,206.101595,389.990328,378.921986,570.428117
min,1.000000,1.000000,2010.000000,1.00000,8665.000000,-5.000000,-155.110000,5.395000,6.912000,6.912000,523.200000,681.800000,302.300000,1367.400000,371.800000,708.700000,736.900000,1729.700000
25%,4.000000,8.000000,2012.000000,6.75000,12636.630000,26.570000,24.200000,37.746000,64.281750,64.283500,1121.200000,1143.400000,558.900000,2938.000000,784.100000,1413.300000,1683.100000,2473.500000
50%,7.000000,16.000000,2014.000000,12.50000,14397.000000,36.180000,34.470000,49.502000,76.038000,76.038000,1333.050000,1329.600000,648.800000,3444.450000,916.600000,1659.400000,1951.700000,2869.500000
75%,10.000000,23.000000,2016.000000,18.25000,16122.000000,48.680000,47.760000,60.218000,86.303000,86.295250,1485.400000,1450.100000,721.500000,3915.700000,1032.300000,1887.400000,2199.125000,3211.100000
max,12.000000,31.000000,2018.000000,24.00000,27762.000000,785.000000,2493.150000,118.547000,168.430000,168.430000,2433.300000,2034.100000,1002.500000,7218.800000,4314.300000,3644.700000,3652.300000,5657.900000


In [25]:
# Shape of dataframe
data.shape

(78840, 19)

In [26]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.2.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [27]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.2.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.2.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [28]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [29]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59130, 24)
Training Target Shape: (59130,)
Testing Predictors Shape: (19710, 24)
Testing Target Shape: (19710,)


In [30]:
# Create scaler
scaler = StandardScaler()

In [31]:
# Scale training predictors
train_pred_scaled = scaler.fit_transform(train_predictors)

In [32]:
# Scale test predictors
test_pred_scaled = scaler.transform(test_predictors)

In [33]:
# Create KNN model
model = KNeighborsRegressor()

In [34]:
# Train model with scaled training data and target values
model.fit(train_pred_scaled, train_target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [35]:
# Calculate errors for training data using mean squared error (MSE) and mean absolute error (MAE)
mse = mean_squared_error(train_target, model.predict(train_pred_scaled))
mae = mean_absolute_error(train_target, model.predict(train_pred_scaled))

In [36]:
# Display training MSE, MAE, & RMSE
print("mse = ",mse," & mae = ",mae," & rmse = ", sqrt(mse))

mse =  21.127929817722304  & mae =  2.9179787180788095  & rmse =  4.596512788813092


In [37]:
# Calculate and display MSE, MAE, & RMSE on test data
test_mse = mean_squared_error(test_target, model.predict(test_pred_scaled))
test_mae = mean_absolute_error(test_target, model.predict(test_pred_scaled))
print("mse = ",test_mse," & mae = ",test_mae," & rmse = ", sqrt(test_mse))

mse =  34.68404177490411  & mae =  3.8109595332318618  & rmse =  5.889315900416967


In [38]:
# Use predict method on the test data
predictions = model.predict(test_pred_scaled)

# Calculate max and mean test_target value
test_target_max_2 = max(test_target)
test_target_mean_2 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_2 = max(errors)
mean_absolute_error_2 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_2 = 100 * (errors / test_target)
accuracy_2 = 100 - np.mean(mape_2)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_2, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_2, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_2, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_2, 2), 'MW')
print('Accuracy:', round(accuracy_2, 2), '%.')

Target Variable Max Value: 168.43 MW
Max Absolute Error: 50.11 MW
Mean Target Variable Value: 76.92 MW
Mean Absolute Error: 3.81 MW
Accuracy: 95.08 %.


In [39]:
# Grid search for KNN to see if performance can be improved
grid_params = {
    'n_neighbors': [3,5,11,19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

gs = GridSearchCV(
     KNeighborsRegressor(),
     grid_params,
     verbose = 1,
     cv = 3,
     n_jobs = -1
)

gs_knn = gs.fit(train_pred_scaled, train_target)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 20.9min finished


In [40]:
gs_knn.best_score_

0.889224205775654

In [41]:
gs_knn.best_estimator_

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='manhattan',
          metric_params=None, n_jobs=None, n_neighbors=3, p=2,
          weights='distance')

In [42]:
gs_knn.best_params_

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}

In [43]:
# See if the grid search improved the accuracy of KNN
best_grid = gs_knn.best_estimator_

# Use predict method on the test data
predictions = best_grid.predict(test_pred_scaled)

# Calculate the absolute errors
errors = abs(predictions - test_target)

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_target)

# Calculate accuracy
accuracy = 100 - np.mean(mape)

# Print out the mean absolute error (MAE) and accuracy
print('Mean Absolute Error:', round(np.mean(errors), 2), 'MW')
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 3.11 MW
Accuracy: 95.93 %.


In [44]:
# See if the grid search improved the accuracy of KNN
best_grid = gs_knn.best_estimator_

# Use predict method on the test data
predictions = best_grid.predict(test_pred_scaled)

# Calculate max and mean test_target value
test_target_max_GS_2 = max(test_target)
test_target_mean_GS_2 = np.mean(test_target)

# Calculate the absolute errors
errors = abs(predictions - test_target)
max_absolute_error_GS_2 = max(errors)
mean_absolute_error_GS_2 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_GS_2 = 100 * (errors / test_target)
accuracy_GS_2 = 100 - np.mean(mape_GS_2)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_GS_2, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_GS_2, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_GS_2, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_GS_2, 2), 'MW')
print('Accuracy:', round(accuracy_GS_2, 2), '%.')

Target Variable Max Value: 168.43 MW
Max Absolute Error: 50.5 MW
Mean Target Variable Value: 76.92 MW
Mean Absolute Error: 3.11 MW
Accuracy: 95.93 %.


### Predict UES load - 3 Days Ahead

In [45]:
# Read in data
data = pd.read_csv("ISO.aggregate - 3 day SHIFTED.csv")

In [46]:
# Shape of dataframe
data.shape

(78816, 19)

In [47]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.3.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [48]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.3.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.3.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [49]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [50]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59112, 24)
Training Target Shape: (59112,)
Testing Predictors Shape: (19704, 24)
Testing Target Shape: (19704,)


In [51]:
# Create scaler
scaler = StandardScaler()

# Scale training predictors
train_pred_scaled = scaler.fit_transform(train_predictors)

# Scale test predictors
test_pred_scaled = scaler.transform(test_predictors)

# Create KNN model
model = KNeighborsRegressor()

# Train model with scaled training data and target values
model.fit(train_pred_scaled, train_target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [52]:
# Calculate errors for training data using mean squared error (MSE) and mean absolute error (MAE)
mse = mean_squared_error(train_target, model.predict(train_pred_scaled))
mae = mean_absolute_error(train_target, model.predict(train_pred_scaled))

In [53]:
# Display training MSE, MAE, & RMSE
print("mse = ",mse," & mae = ",mae," & rmse = ", sqrt(mse))

mse =  24.543032332639733  & mae =  3.1042531161185547  & rmse =  4.9540924832546


In [54]:
# Calculate and display MSE, MAE, & RMSE on test data
test_mse = mean_squared_error(test_target, model.predict(test_pred_scaled))
test_mae = mean_absolute_error(test_target, model.predict(test_pred_scaled))
print("mse = ",test_mse," & mae = ",test_mae," & rmse = ", sqrt(test_mse))

mse =  41.65758531262891  & mae =  4.159194346325618  & rmse =  6.454268766686813


In [55]:
# Use predict method on the test data
predictions = model.predict(test_pred_scaled)

# Calculate max and mean test_target value
test_target_max_3 = max(test_target)
test_target_mean_3 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_3 = max(errors)
mean_absolute_error_3 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_3 = 100 * (errors / test_target)
accuracy_3 = 100 - np.mean(mape_3)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_3, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_3, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_3, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_3, 2), 'MW')
print('Accuracy:', round(accuracy_3, 2), '%.')

Target Variable Max Value: 168.42 MW
Max Absolute Error: 61.81 MW
Mean Target Variable Value: 77.07 MW
Mean Absolute Error: 4.16 MW
Accuracy: 94.62 %.


### Predict UES load - 4 Days Ahead

In [56]:
# Read in data
data = pd.read_csv("ISO.aggregate - 4 day SHIFTED.csv")

# Shape of dataframe
data.shape

(78792, 19)

In [57]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.4.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [58]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.4.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.4.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [59]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [60]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59094, 24)
Training Target Shape: (59094,)
Testing Predictors Shape: (19698, 24)
Testing Target Shape: (19698,)


In [61]:
# Create scaler
scaler = StandardScaler()

# Scale training predictors
train_pred_scaled = scaler.fit_transform(train_predictors)

# Scale test predictors
test_pred_scaled = scaler.transform(test_predictors)

# Create KNN model
model = KNeighborsRegressor()

# Train model with scaled training data and target values
model.fit(train_pred_scaled, train_target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [62]:
# Calculate errors for training data using mean squared error (MSE) and mean absolute error (MAE)
mse = mean_squared_error(train_target, model.predict(train_pred_scaled))
mae = mean_absolute_error(train_target, model.predict(train_pred_scaled))

# Display training MSE, MAE, & RMSE
print("mse = ",mse," & mae = ",mae," & rmse = ", sqrt(mse))

mse =  25.638946062052323  & mae =  3.1887020306630114  & rmse =  5.063491489284082


In [63]:
# Calculate and display MSE, MAE, & RMSE on test data
test_mse = mean_squared_error(test_target, model.predict(test_pred_scaled))
test_mae = mean_absolute_error(test_target, model.predict(test_pred_scaled))
print("mse = ",test_mse," & mae = ",test_mae," & rmse = ", sqrt(test_mse))

mse =  42.37801377043355  & mae =  4.24092216468677  & rmse =  6.509839765342427


In [64]:
# Use predict method on the test data
predictions = model.predict(test_pred_scaled)

# Calculate max and mean test_target value
test_target_max_4 = max(test_target)
test_target_mean_4 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_4 = max(errors)
mean_absolute_error_4 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_4 = 100 * (errors / test_target)
accuracy_4 = 100 - np.mean(mape_4)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_4, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_4, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_4, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_4, 2), 'MW')
print('Accuracy:', round(accuracy_4, 2), '%.')

Target Variable Max Value: 168.43 MW
Max Absolute Error: 58.35 MW
Mean Target Variable Value: 77.13 MW
Mean Absolute Error: 4.24 MW
Accuracy: 94.51 %.


### Predict UES load - 5 Days Ahead

In [65]:
# Read in data
data = pd.read_csv("ISO.aggregate - 5 day SHIFTED.csv")

# Shape of dataframe
data.shape

(78768, 19)

In [66]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.5.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [67]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.5.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.5.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [68]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [69]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59076, 24)
Training Target Shape: (59076,)
Testing Predictors Shape: (19692, 24)
Testing Target Shape: (19692,)


In [70]:
# Create scaler
scaler = StandardScaler()

# Scale training predictors
train_pred_scaled = scaler.fit_transform(train_predictors)

# Scale test predictors
test_pred_scaled = scaler.transform(test_predictors)

# Create KNN model
model = KNeighborsRegressor()

# Train model with scaled training data and target values
model.fit(train_pred_scaled, train_target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [71]:
# Calculate errors for training data using mean squared error (MSE) and mean absolute error (MAE)
mse = mean_squared_error(train_target, model.predict(train_pred_scaled))
mae = mean_absolute_error(train_target, model.predict(train_pred_scaled))

# Display training MSE, MAE, & RMSE
print("mse = ",mse," & mae = ",mae," & rmse = ", sqrt(mse))

mse =  25.819442819207797  & mae =  3.208577249644526  & rmse =  5.081283579884889


In [72]:
# Calculate and display MSE, MAE, & RMSE on test data
test_mse = mean_squared_error(test_target, model.predict(test_pred_scaled))
test_mae = mean_absolute_error(test_target, model.predict(test_pred_scaled))
print("mse = ",test_mse," & mae = ",test_mae," & rmse = ", sqrt(test_mse))

mse =  43.806373735267115  & mae =  4.298805809465773  & rmse =  6.618638359607444


In [73]:
# Use predict method on the test data
predictions = model.predict(test_pred_scaled)

# Calculate max and mean test_target value
test_target_max_5 = max(test_target)
test_target_mean_5 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_5 = max(errors)
mean_absolute_error_5 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_5 = 100 * (errors / test_target)
accuracy_5 = 100 - np.mean(mape_5)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_5, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_5, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_5, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_5, 2), 'MW')
print('Accuracy:', round(accuracy_5, 2), '%.')

Target Variable Max Value: 165.69 MW
Max Absolute Error: 56.37 MW
Mean Target Variable Value: 77.08 MW
Mean Absolute Error: 4.3 MW
Accuracy: 94.4 %.


### Predict UES load - 6 Days Ahead

In [74]:
# Read in data
data = pd.read_csv("ISO.aggregate - 6 day SHIFTED.csv")

# Shape of dataframe
data.shape

(78744, 19)

In [75]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.6.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [76]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.6.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.6.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [77]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [78]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59058, 24)
Training Target Shape: (59058,)
Testing Predictors Shape: (19686, 24)
Testing Target Shape: (19686,)


In [79]:
# Create scaler
scaler = StandardScaler()

# Scale training predictors
train_pred_scaled = scaler.fit_transform(train_predictors)

# Scale test predictors
test_pred_scaled = scaler.transform(test_predictors)

# Create KNN model
model = KNeighborsRegressor()

# Train model with scaled training data and target values
model.fit(train_pred_scaled, train_target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [80]:
# Calculate errors for training data using mean squared error (MSE) and mean absolute error (MAE)
mse = mean_squared_error(train_target, model.predict(train_pred_scaled))
mae = mean_absolute_error(train_target, model.predict(train_pred_scaled))

# Display training MSE, MAE, & RMSE
print("mse = ",mse," & mae = ",mae," & rmse = ", sqrt(mse))

mse =  26.195328832698703  & mae =  3.1905566663280163  & rmse =  5.118137242464167


In [81]:
# Calculate and display MSE, MAE, & RMSE on test data
test_mse = mean_squared_error(test_target, model.predict(test_pred_scaled))
test_mae = mean_absolute_error(test_target, model.predict(test_pred_scaled))
print("mse = ",test_mse," & mae = ",test_mae," & rmse = ", sqrt(test_mse))

mse =  42.2152503732033  & mae =  4.207828446611805  & rmse =  6.497326401928973


In [82]:
# Use predict method on the test data
predictions = model.predict(test_pred_scaled)

# Calculate max and mean test_target value
test_target_max_6 = max(test_target)
test_target_mean_6 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_6 = max(errors)
mean_absolute_error_6 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_6 = 100 * (errors / test_target)
accuracy_6 = 100 - np.mean(mape_6)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_6, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_6, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_6, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_6, 2), 'MW')
print('Accuracy:', round(accuracy_6, 2), '%.')

Target Variable Max Value: 166.73 MW
Max Absolute Error: 62.63 MW
Mean Target Variable Value: 76.96 MW
Mean Absolute Error: 4.21 MW
Accuracy: 94.49 %.


### Predict UES load - 7 Days Ahead

In [83]:
# Read in data
data = pd.read_csv("ISO.aggregate - 7 day SHIFTED.csv")

# Shape of dataframe
data.shape

(78720, 19)

In [84]:
# One-hot encoding
data = pd.get_dummies(data)

# Check dataframe data types
data.dtypes

Month                     int64
Day                       int64
Year                      int64
TI                        int64
ISO.tot.dmd             float64
LMP.DA.NH               float64
LMP.RT.NH               float64
UES.Cap                 float64
UES.Sea                 float64
UES.Sea.7.DAYS.AHEAD    float64
NH.RT.MWh               float64
ME.RT.MWh               float64
VT.RT.MWh               float64
CT.RT.MWh               float64
RI.RT.MWh               float64
MA.SE.RT.MWh            float64
MA.WC.RT.MWh            float64
MA.NE.RT.MWh            float64
Wk.Day_Fri                uint8
Wk.Day_Mon                uint8
Wk.Day_Sat                uint8
Wk.Day_Sun                uint8
Wk.Day_Thu                uint8
Wk.Day_Tue                uint8
Wk.Day_Wed                uint8
dtype: object

In [85]:
# Split data into target variable and predictor variables

# Target is the value we want to predict (in this case, MW of UES-Seacoast Load Region)
target = np.array(data['UES.Sea.7.DAYS.AHEAD'])

# Remove the target from the predictors
# axis 1 refers to the columns
predictors = data.drop('UES.Sea.7.DAYS.AHEAD', axis = 1)

# Saving predictor names for later use
predictor_list = list(predictors.columns)

# Convert to numpy array
predictors = np.array(predictors)

In [86]:
# Split the data into training and testing sets using sklearn
train_predictors, test_predictors, train_target, test_target = train_test_split(predictors, target, test_size = 0.25, random_state = 77)


In [87]:
# Check shape of train and test data
print('Training Predictors Shape:', train_predictors.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Predictors Shape:', test_predictors.shape)
print('Testing Target Shape:', test_target.shape)

Training Predictors Shape: (59040, 24)
Training Target Shape: (59040,)
Testing Predictors Shape: (19680, 24)
Testing Target Shape: (19680,)


In [88]:
# Create scaler
scaler = StandardScaler()

# Scale training predictors
train_pred_scaled = scaler.fit_transform(train_predictors)

# Scale test predictors
test_pred_scaled = scaler.transform(test_predictors)

# Create KNN model
model = KNeighborsRegressor()

# Train model with scaled training data and target values
model.fit(train_pred_scaled, train_target)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [89]:
# Calculate errors for training data using mean squared error (MSE) and mean absolute error (MAE)
mse = mean_squared_error(train_target, model.predict(train_pred_scaled))
mae = mean_absolute_error(train_target, model.predict(train_pred_scaled))

# Display training MSE, MAE, & RMSE
print("mse = ",mse," & mae = ",mae," & rmse = ", sqrt(mse))

mse =  26.23847870173645  & mae =  3.1719358773712734  & rmse =  5.122350895998482


In [90]:
# Calculate and display MSE, MAE, & RMSE on test data
test_mse = mean_squared_error(test_target, model.predict(test_pred_scaled))
test_mae = mean_absolute_error(test_target, model.predict(test_pred_scaled))
print("mse = ",test_mse," & mae = ",test_mae," & rmse = ", sqrt(test_mse))

mse =  43.15694502939837  & mae =  4.225755955284553  & rmse =  6.569394570993462


In [91]:
# Use predict method on the test data
predictions = model.predict(test_pred_scaled)

# Calculate max and mean test_target value
test_target_max_7 = max(test_target)
test_target_mean_7 = np.mean(test_target)

# Calculate the absolute errors, calculate max and mean absolute error
errors = abs(predictions - test_target)
max_absolute_error_7 = max(errors)
mean_absolute_error_7 = np.mean(errors)

# Calculate mean absolute percentage error (MAPE), and Accuracy
mape_7 = 100 * (errors / test_target)
accuracy_7 = 100 - np.mean(mape_7)

# Print out some statistics, including accuracy
print('Target Variable Max Value:', round(test_target_max_7, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_7, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_7, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_7, 2), 'MW')
print('Accuracy:', round(accuracy_7, 2), '%.')

Target Variable Max Value: 168.42 MW
Max Absolute Error: 55.71 MW
Mean Target Variable Value: 77.03 MW
Mean Absolute Error: 4.23 MW
Accuracy: 94.54 %.


##### Compare Performance of the 7 KNN Models

In [92]:
### Print out performance metrics from the seven Random Forest Models

print('KNN Model 1 - Predict UES Load 1 Day Ahead')
print('Target Variable Max Value:', round(test_target_max_1, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_1, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_1, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_1, 2), 'MW')
print('Accuracy:', round(accuracy_1, 2), '%.')
print('')

print('KNN Model 2 - Predict UES Load 2 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_2, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_2, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_2, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_2, 2), 'MW')
print('Accuracy:', round(accuracy_2, 2), '%.')
print('')

print('KNN Model 3 - Predict UES Load 3 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_3, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_3, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_3, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_3, 2), 'MW')
print('Accuracy:', round(accuracy_3, 2), '%.')
print('')

print('KNN Model 4 - Predict UES Load 4 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_4, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_4, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_4, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_4, 2), 'MW')
print('Accuracy:', round(accuracy_4, 2), '%.')
print('')

print('KNN Model 5 - Predict UES Load 5 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_5, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_5, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_5, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_5, 2), 'MW')
print('Accuracy:', round(accuracy_5, 2), '%.')
print('')

print('KNN Model 6 - Predict UES Load 6 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_6, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_6, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_6, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_6, 2), 'MW')
print('Accuracy:', round(accuracy_6, 2), '%.')
print('')

print('KNN Model 7 - Predict UES Load 7 Days Ahead')
print('Target Variable Max Value:', round(test_target_max_7, 2), 'MW')
print('Max Absolute Error:', round(max_absolute_error_7, 2), 'MW')
print('Mean Target Variable Value:', round(test_target_mean_7, 2), 'MW')
print('Mean Absolute Error:', round(mean_absolute_error_7, 2), 'MW')
print('Accuracy:', round(accuracy_7, 2), '%.')
print('')

KNN Model 1 - Predict UES Load 1 Day Ahead
Target Variable Max Value: 166.65 MW
Max Absolute Error: 42.61 MW
Mean Target Variable Value: 77.05 MW
Mean Absolute Error: 3.05 MW
Accuracy: 96.09 %.

KNN Model 2 - Predict UES Load 2 Days Ahead
Target Variable Max Value: 168.43 MW
Max Absolute Error: 50.11 MW
Mean Target Variable Value: 76.92 MW
Mean Absolute Error: 3.81 MW
Accuracy: 95.08 %.

KNN Model 3 - Predict UES Load 3 Days Ahead
Target Variable Max Value: 168.42 MW
Max Absolute Error: 61.81 MW
Mean Target Variable Value: 77.07 MW
Mean Absolute Error: 4.16 MW
Accuracy: 94.62 %.

KNN Model 4 - Predict UES Load 4 Days Ahead
Target Variable Max Value: 168.43 MW
Max Absolute Error: 58.35 MW
Mean Target Variable Value: 77.13 MW
Mean Absolute Error: 4.24 MW
Accuracy: 94.51 %.

KNN Model 5 - Predict UES Load 5 Days Ahead
Target Variable Max Value: 165.69 MW
Max Absolute Error: 56.37 MW
Mean Target Variable Value: 77.08 MW
Mean Absolute Error: 4.3 MW
Accuracy: 94.4 %.

KNN Model 6 - Predict U

In [93]:
# Create a dataframe of performance metrics of 7 models
model_number = [1,2,3,4,5,6,7]

test_target_max_values = [test_target_max_1, test_target_max_2, test_target_max_3, test_target_max_4, 
                          test_target_max_6, test_target_max_6, test_target_max_7]

max_absolute_error_values =[max_absolute_error_1, max_absolute_error_2, max_absolute_error_3, max_absolute_error_4, 
                            max_absolute_error_5, max_absolute_error_6, max_absolute_error_7]

test_target_mean_values = [test_target_mean_1, test_target_mean_2, test_target_mean_3, test_target_mean_4, 
                           test_target_mean_5, test_target_mean_6, test_target_mean_7]

mean_absolute_error_values = [mean_absolute_error_1, mean_absolute_error_2, mean_absolute_error_3, 
                              mean_absolute_error_4, mean_absolute_error_5, mean_absolute_error_6, 
                              mean_absolute_error_7]

accuracy_values = [accuracy_1, accuracy_2, accuracy_3, accuracy_4, accuracy_5, accuracy_6, accuracy_7]

### Create pandas dataframe using metrics
metrics_df = {'Model Number': model_number,
              'Test Target Max': test_target_max_values, 
              'Max Absolute Error': max_absolute_error_values,
              'Test Target Mean': test_target_mean_values,
              'Mean Absolute Error': mean_absolute_error_values,
              'Accuracy': accuracy_values}

metrics_df = pd.DataFrame(metrics_df)
metrics_df


,Model Number,Test Target Max,Max Absolute Error,Test Target Mean,Mean Absolute Error,Accuracy
0,1,166.649,42.6108,77.047343,3.054505,96.091557
1,2,168.430,50.1132,76.918785,3.810960,95.075031
2,3,168.420,61.8070,77.070027,4.159194,94.622248
3,4,168.430,58.3490,77.126991,4.240922,94.506124
4,5,166.733,56.3736,77.079134,4.298806,94.402303
5,6,166.733,62.6306,76.964268,4.207828,94.489007
6,7,168.420,55.7116,77.033948,4.225756,94.542289


In [94]:
# Write out a .csv of the performance metrics
# metrics_df.to_csv('metrics_KNN.csv', index=False)